<a href="https://colab.research.google.com/github/nlinh2017/Cinema4D-Exporter/blob/master/GWAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("googledrive")
library("googledrive")

if (file.exists("/usr/local/lib/python3.6/dist-packages/google/colab/_ipython.py")){ 
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check)
  options(rlang_interactive=TRUE)
}                                 


In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("GWASTools")

In [ ]:
library(GWASTools)
library(SNPRelate)
bed.fn <- system.file("extdata", "plinkhapmap.bed.gz", package="SNPRelate")
fam.fn <- system.file("extdata", "plinkhapmap.fam.gz", package="SNPRelate")
bim.fn <- system.file("extdata", "plinkhapmap.bim.gz", package="SNPRelate")
gdsfile <- "snps.gds"
snpgdsBED2GDS(bed.fn, fam.fn, bim.fn, gdsfile, family=TRUE,
              cvt.chr="int", cvt.snpid="int", verbose=FALSE)

In [ ]:
 library(SNPRelate)

In [ ]:
bed.fn <- system.file("extdata", "plinkhapmap.bed.gz", package="SNPRelate")

In [ ]:
bed.fn <- system.file("extdata", "plinkhapmap.bed.gz", package="SNPRelate")
fam.fn <- system.file("extdata", "plinkhapmap.fam.gz", package="SNPRelate")
bim.fn <- system.file("extdata", "plinkhapmap.bim.gz", package="SNPRelate")
gdsfile <- "snps.gds"

In [ ]:
snpgdsBED2GDS(bed.fn, fam.fn, bim.fn, gdsfile, family=TRUE, cvt.chr="int", cvt.snpid="int", verbose=FALSE)

In [ ]:
gdsfile

In [ ]:
library(GWASTools)


In [ ]:
snpgdsBED2GDS(bed.fn, fam.fn, bim.fn, gdsfile, family=TRUE, cvt.chr="int", cvt.snpid="int", verbose=FALSE)